# STIPS Advanced Tutorial I – Further Observations (Noise and Distortion)

The contents of this notebook assume both that you already have STIPS installed (see [Installing STIPS](https://stsci-stips.readthedocs.io/en/latest/installation.html) if not) and that you are comfortable with basic usage of STIPS functionalities (see [STIPS Basic Tutorial](https://stips.readthedocs.io/en/latest/basic_tutorial.html), or the Basic Tutorial notebook, if not).  Procedures in this tutorial will use scenes generated by the Basic Tutorial to explore further STIPS functionalities.

## Checking STIPS Import

Before beginning, check again that the STIPS import is correct and configure and import and configure pyplot.

In [ ]:
from astropy.io import fits
from matplotlib import style
from stips.observation_module import ObservationModule
from stips.scene_module import SceneModule
import matplotlib
import matplotlib.pyplot as plt
import stips

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'

print(stips.__env__report__)

## Generating a Smaller Scene

To better see the scope of possible observations, we will generate a scene smaller in scale than in the Basic Tutorial –– we do that below, following the same procedure as before.  The specifications of our scene are as follows:

* A stellar population representing a globular cluster with
  * 100 stars
  * An age of 7.5 billion years
  * A metallicity of -2.0
  * A Salpeter IMF with alpha=-2.35
  * A binary fraction of 10%
  * A clustered distribution (higher-mass stars closer to the population centre)
  * An inverse power-law distribution
  * A radius of 100 parsecs
  * A distance of 10 kpc
  * No offset from the centre of the scene being created
* A collection of background galaxies with
  * 2 galaxies
  * Redshifts between 0 and 0.2
  * Radii between 0.01 and 2.0 arcsec
  * V-band surface brightness magnitudes between 28 and 25
  * Uniform spatial distribution (unclustered) over 200 arcsec
  * No offset from the centre of the scene being created

In [ ]:
obs_prefix = 'adv_notebook'
obs_ra = 150.0
obs_dec = -2.5

In [ ]:
scm = SceneModule(out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec, out_path='notebooks_data/')

stellar_parameters = {
                      'n_stars': 100,
                      'age_low': 7.5e12, 
                      'age_high': 7.5e12,
                      'z_low': -2.0, 
                      'z_high': -2.0,
                      'imf': 'salpeter', 
                      'alpha': -2.35,
                      'binary_fraction': 0.1,
                      'clustered': True,
                      'distribution': 'invpow',
                      'radius': 100.0, 
                      'radius_units': 'pc',
                      'distance_low': 10.0, 
                      'distance_high': 10.0,
                      'offset_ra': 0.0, 
                      'offset_dec': 0.0
                     }

stellar_cat_file = scm.CreatePopulation(stellar_parameters)
print("Stellar population saved to file {}".format(stellar_cat_file))

galaxy_parameters = {
                     'n_gals': 2,
                     'z_low': 0.0, 
                     'z_high': 0.2,
                     'rad_low': 0.01, 
                     'rad_high': 2.0,
                     'sb_v_low': 28.0, 
                     'sb_v_high': 25.0,
                     'distribution': 'uniform', 
                     'clustered': False,
                     'radius': 200.0, 
                     'radius_units': 'arcsec',
                     'offset_ra': 0.0, 
                     'offset_dec': 0.0
                    }

galaxy_cat_file = scm.CreateGalaxies(galaxy_parameters)
print("Galaxy population saved to file {}".format(galaxy_cat_file))

## Observations

With the scene now created, we can move forward with our observations.  For the purpose of seeing multiple observation styles, we'll run through a series of observations, with the following parameters.  This is meant to demonstrate the range of STIPS observation capabilities –– there are, of course, a great many possibilities for other observations, and this is only a small sample of options.  We will leave all other parameters unchanged beyond the specified modifications in each observation.  We will begin with the observation setup used in the Basic Tutorial, and compare other observations against that.

### Observation I: Offset Modifications

* Offset of 10 degrees in RA
* Rotation of 27 degrees

### Observation II: Other Residuals

* Flat residuals –– True
* Dark residauls –– True
* Cosmic residuals –– True
* Poisson residuals –– False
* Readnoise residuals –– False

### Observation III: Distortions and Exposures

#### III.1

* Introduce distortion

#### III.2

* Increase exposure time to 5000

#### III.3 

* Decrease exposure time to 100

## Basic Tutorial Observation

In [ ]:
offset = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }

residuals = {
             'residual_flat': False,
             'residual_dark': False,
             'residual_cosmic': False,
             'residual_poisson': True,
             'residual_readnoise': True,
            }

observation_parameters = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 1000,
                          'offsets': [offset]
                         }

obm = ObservationModule(observation_parameters, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec,
                        residuals=residuals, out_path='notebooks_data/')

obm.nextObservation()

output_stellar_catalogues = obm.addCatalogue(stellar_cat_file)
output_galaxy_catalogues = obm.addCatalogue(galaxy_cat_file)

psf_file = obm.addError()

fits_file, mosaic_file, params = obm.finalize(mosaic=False)

In [ ]:
with fits.open(fits_file) as result_file:
    result_data = result_file[1].data
    
crop_result_data = result_data[1900:2200, 1900:2200]
norm = matplotlib.colors.LogNorm(vmin=crop_result_data.min(), vmax=crop_result_data.max())

fig1 = plt.figure()
im = plt.matshow(crop_result_data, norm=norm)

## Observation I

In [ ]:
offset_1 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 40.0,
          'offset_dec': 20.0,
          'offset_pa': 27.0
         }


residuals_1 = {
             'residual_flat': False,
             'residual_dark': False,
             'residual_cosmic': False,
             'residual_poisson': True,
             'residual_readnoise': True
            }


observation_parameters_1 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 1000,
                          'offsets': [offset]
                         }

obm_1 = ObservationModule(observation_parameters_1, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec,
                          residual=residuals_1, out_path='notebooks_data/')

obm_1.nextObservation()

In [ ]:
output_stellar_catalogues_1 = obm_1.addCatalogue(stellar_cat_file)
output_galaxy_catalogues_1 = obm_1.addCatalogue(galaxy_cat_file)

psf_file_1 = obm_1.addError()

fits_file_1, mosaic_file_1, params_1 = obm_1.finalize(mosaic=False)

In [ ]:
with fits.open(fits_file_1) as result_file_1:
    result_data_1 = result_file_1[1].data

crop_result_data_1 = result_data_1[1900:2200, 1900:2200]
norm = matplotlib.colors.LogNorm(vmin=crop_result_data_1.min(), vmax=crop_result_data_1.max())

fig_1 = plt.figure()
im_1 = plt.matshow(crop_result_data_1, norm=norm)

## Observation II

In [ ]:
offset_2 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }


residuals_2 = {
             'residual_flat': True,
             'residual_dark': True,
             'residual_cosmic': True,
             'residual_poisson': False,
             'residual_readnoise': False
            }


observation_parameters_2 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 1000,
                          'offsets': [offset]
                         }

obm_2 = ObservationModule(observation_parameters_2, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec,
                          residual=residuals_2, out_path='notebooks_data/')

obm_2.nextObservation()

In [ ]:
output_stellar_catalogues_2 = obm_2.addCatalogue(stellar_cat_file)
output_galaxy_catalogues_2 = obm_2.addCatalogue(galaxy_cat_file)

psf_file_2 = obm_2.addError()

fits_file_2, mosaic_file_2, params_2 = obm_2.finalize(mosaic=False)

In [ ]:
with fits.open(fits_file_2) as result_file_2:
    result_data_2 = result_file_2[1].data

crop_result_data_2 = result_data[1900:2200, 1900:2200]
norm = matplotlib.colors.LogNorm(vmin=crop_result_data_2.min(), vmax=crop_result_data_2.max())

fig_2 = plt.figure()
im_2 = plt.matshow(crop_result_data_2, norm=norm)

## Observation III.1

In [ ]:
offset_3_1 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }


residuals_3_1 = {
             'residual_flat': False,
             'residual_dark': False,
             'residual_cosmic': False,
             'residual_poisson': True,
             'residual_readnoise': True
            }


observation_parameters_3_1 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': True,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 1000,
                          'offsets': [offset]
                         }

obm_3_1 = ObservationModule(observation_parameters_3_1, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec,
                            residual=residuals_3_1, out_path='notebooks_data/')

obm_3_1.nextObservation()

In [ ]:
output_stellar_catalogues_3_1 = obm_3_1.addCatalogue(stellar_cat_file)
output_galaxy_catalogues_3_1 = obm_3_1.addCatalogue(galaxy_cat_file)

psf_file_3_1 = obm_3_1.addError()

fits_file_3_1, mosaic_file_3_1, params_3_1 = obm_3_1.finalize(mosaic=False)

In [ ]:
with fits.open(fits_file_3_1) as result_file_3_1:
    result_data_3_1 = result_file_3_1[1].data

crop_result_data_3_1 = result_data_3_1 [1900:2200, 1900:2200]
norm = matplotlib.colors.LogNorm(vmin=crop_result_data_3_1.min(), vmax=crop_result_data_3_1.max())

fig_3_1 = plt.figure()
im_3_1 = plt.matshow(crop_result_data_3_1, norm=norm)

## Observation III.2

In [ ]:
offset_3_2 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }


residuals_3_2 = {
             'residual_flat': False,
             'residual_dark': False,
             'residual_cosmic': False,
             'residual_poisson': False,
             'residual_readnoise': False
            }


observation_parameters_3_2 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 15000,
                          'offsets': [offset]
                         }

obm_3_2 = ObservationModule(observation_parameters, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec,
                            residual=residuals, out_path='notebooks_data/')

obm_3_2.nextObservation()

In [ ]:
output_stellar_catalogues_3_2 = obm_3_2.addCatalogue(stellar_cat_file)
output_galaxy_catalogues_3_2 = obm_3_2.addCatalogue(galaxy_cat_file)

psf_file_3_2 = obm_3_2.addError()

fits_file_3_2, mosaic_file_3_2, params_3_2 = obm_3_2.finalize(mosaic=False)

In [ ]:
with fits.open(fits_file_3_2) as result_file_3_2:
    result_data_3_2 = result_file_3_2[1].data

crop_result_data_3_2 = result_data_3_2[1900:2200, 1900:2200]
norm = matplotlib.colors.LogNorm(vmin=crop_result_data_3_2.min(), vmax=crop_result_data_3_2.max())

fig_3_2 = plt.figure()
im_3_2 = plt.matshow(crop_result_data_3_2, norm=norm)

## Observation III.3

In [ ]:
offset_3_3 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }


residuals_3_3 = {
             'residual_flat': False,
             'residual_dark': False,
             'residual_cosmic': False,
             'residual_poisson': False,
             'residual_readnoise': False
            }


observation_parameters_3_3 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 100,
                          'offsets': [offset]
                         }

obm_3_3 = ObservationModule(observation_parameters, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec,
                            residual=residuals, out_path='notebooks_data/')

obm_3_3.nextObservation()

In [ ]:
output_stellar_catalogues_3_3 = obm_3_3.addCatalogue(stellar_cat_file)
output_galaxy_catalogues_3_3 = obm_3_3.addCatalogue(galaxy_cat_file)

psf_file_3_3 = obm_3_3.addError()

fits_file_3_3, mosaic_file_3_3, params_3_3 = obm_3_3.finalize(mosaic=False)

In [ ]:
with fits.open(fits_file_3_3) as result_file_3_3:
    result_data_3_3 = result_file_3_3[1].data

crop_result_data_3_3 = result_data_3_3[1900:2200, 1900:2200]
norm = matplotlib.colors.LogNorm(vmin=crop_result_data_3_3.min(), vmax=crop_result_data_3_3.max())

fig_3_3 = plt.figure()
im_3_3 = plt.matshow(crop_result_data_3_3, norm = norm)

## Conclusion

This concludes the first portion of the advanced tutorial.  For information on PSFs, please see STIPS Advanced II.  If you have further questions, check out the [STIPS documentation](https://stips.readthedocs.io/en/latest/), or reach out to the STIPS Helpdesk via email at help@stsci.edu with the subject line "STIPS Question".